In [ ]:
!pip uninstall -y edge-rec
!pip install -e.

In [1]:
from edge_rec.gaussian_diffusion import GaussianDiffusion, Trainer
from edge_rec.model import GraphReconstructionModel

model = GraphReconstructionModel.default()

diffusion_model = GaussianDiffusion(model, image_size=(943, 1682))
trainer = Trainer(
    diffusion_model, folder="./movie_lens", train_num_steps=int(1e4),
    full_sample=False, train_on_full_graph=True
)
print("Using device:", trainer.device)

/Users/up/Desktop/CMU/Courses/10-708/project/venv/lib/python3.9/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(split_batches=True)
  warnings.warn(


./movie_lens/processed/data.pt
./movie_lens/processed/data.pt
Using device: mps


In [3]:
from edge_rec.movie_lens_dataset import ProcessedMovieLens

ProcessedMovieLens("./movie_lens", download=False).build_feat_graph().shape

./movie_lens/processed/data.pt


/Users/up/Desktop/CMU/Courses/10-708/project/venv/lib/python3.9/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3588.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


torch.Size([10, 943, 1682])

In [3]:
trainer.train()

  0%|          | 0/10000 [00:00<?, ?it/s]

/Users/up/Desktop/CMU/Courses/10-708/project/venv/lib/python3.9/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3588.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


RuntimeError: MPS backend out of memory (MPS allocated: 35.67 GB, other allocations: 606.80 MB, max allowed: 36.27 GB). Tried to allocate 3.41 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).